In [1]:
import urllib.request
import gzip
import shutil
import os

import numpy as np
import pandas as pd
from hmmlearn.hmm import MultinomialHMM
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation

In this notebook we go through the process of building a hidden markov model (HMM) to approximate the codon use distribution of *Bacillus subtilis*.

First, we download the *B. subtilis genome*. 

In [2]:
url = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/009/045/GCF_000009045.1_ASM904v1/GCF_000009045.1_ASM904v1_genomic.gbff.gz'
gz_fn = 'GCF_000009045.1_ASM904v1_genomic.gbff.gz'
gbk_fn = gz_fn.replace('.gbff.gz', '.gbk')
if not os.path.isfile(gz_fn):
    print('Beginning file download with urllib2...')
    urllib.request.urlretrieve(url, gz_fn)

if not os.path.isfile(gbk_fn):
    with gzip.open(gz_fn, 'rb') as f_in, open(gbk_fn, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

Some helper functions to split a nucleotide sequence into codons and check for ambiguous codons or sequences with lengths that aren't a multiple of 3.

`assert` statements are little tests to make sure that things are working the way we expect. They are *very* helpful for catching silly bugs.

In [3]:
def get_list_of_codons(dna_seq):
    codons = []
    for i in range(0, len(dna_seq), 3):
        codons.append(dna_seq[i:i+3])
    return codons
assert get_list_of_codons('ATGCCCGGGAAATTTTAG') == ['ATG', 'CCC', 'GGG', 'AAA', 'TTT', 'TAG']

def check_len_and_ambiguity(seq):
    assert isinstance(seq, str)
    ambig_nucs = ['R', 'Y', 'S', 'W', 'K', 'M', 'B', 'D', 'H', 'V', 'N']
    unambiguous = not any([anuc in seq for anuc in ambig_nucs])
    multiple_3 = len(seq) % 3 == 0
    return unambiguous and multiple_3
assert check_len_and_ambiguity('ATGACCTAG')
assert not check_len_and_ambiguity('ATGACCTA')
assert not check_len_and_ambiguity('ATGACCTAY')
assert not check_len_and_ambiguity('ATGACCAY')

And now we read in all the protein coding sequences from the genome. During this process, we keep track of the set of unique amino acids and nucleotides, the initial states, and---of great importance---the codons that correspond to each amino acid.

In [4]:
u_aas = set()
u_codons = set()
all_codons = []
initial_states = []
emissions = {}
for record in SeqIO.parse(gbk_fn, "genbank"):
    for feature in record.features:
        if feature.type == 'CDS' and \
           'translation' in feature.qualifiers and \
           check_len_and_ambiguity(str(record.seq)):
            protein = feature.qualifiers['translation'][0] + '*'
            aas = {aa for aa in protein}
            codon = get_list_of_codons(str(feature.extract(record.seq)))
            if len(protein) == len(codon):
                all_codons.append(codon)
                initial_states.append(codon[0])
                u_aas = u_aas.union(aas)
                u_codons = u_codons.union(set(codon))
                for i, cdn in enumerate(codon):
                    emissions[cdn] = protein[i]
lu_aas = list(u_aas)
lu_codons = list(u_codons)

Let's look at some of the objects we created.

In [5]:
lu_codons[:3]

['AAG', 'CAA', 'GCC']

In [6]:
emissions['AAA'], emissions['ATG']

('K', 'M')

Helper functions to encode and decode sequences.

In [7]:
def encode_seq(seq_obj, seqtype='dna'):
    encdr = lu_codons
    symbols = get_list_of_codons(seq_obj)
    if seqtype != 'dna':
        encdr = lu_aas
        symbols = [c for c in seq_obj]
    outseq = np.array([encdr.index(s) for s in symbols])
    return outseq

test_aa = 'MENILD'
test_nuc = 'AAAAAAATAAGATAG'
assert encode_seq(test_aa, seqtype='prot')[0] == lu_aas.index(test_aa[0]) and \
       encode_seq(test_aa, seqtype='prot')[-1] == lu_aas.index(test_aa[-1])
assert encode_seq(test_nuc, seqtype='dna')[0] == lu_codons.index(test_nuc[0:3]) and \
       encode_seq(test_nuc, seqtype='dna')[-1] == lu_codons.index(test_nuc[-3:])

def decode_seq(num_array, seqtype='dna'):
    encdr = lu_codons
    if seqtype != 'dna':
        encdr = lu_aas
    outseq = [encdr[s] for s in num_array]
    return ''.join(outseq)

assert decode_seq(encode_seq(test_nuc)) == test_nuc
assert decode_seq(encode_seq(test_aa, seqtype='prot'), seqtype='prot') == test_aa

Now we construct the three matrices that together form our HMM. 

The `emission_prob` matrix maps the likelihood of a particular codon emitting a particular amino acid. This is pretty simple, since a given codon will map to its amino acid 100% of the time.

In [8]:
emission_prob = np.zeros((len(lu_codons), len(lu_aas)))
for i, codon in enumerate(lu_codons):
    aa = emissions[codon]
    j = lu_aas.index(aa)
    emission_prob[i, j] = 1

The `initial_probabilities` matrix counts the number of times each codon is found at the start of a sequence, then divides those by the total so that they sum to 1. We can verify that "ATG" is a high fraction of the start probability.

In [9]:
initial_probabilities = {k:0 for k in lu_codons}
for i in initial_states:
    initial_probabilities[i] += 1
for k in initial_probabilities.keys():
    initial_probabilities[k] = initial_probabilities[k] / len(initial_states)
initial_probs_np = np.array([initial_probabilities[x] for x in lu_codons])

In [10]:
atg_idx = lu_codons.index('ATG')
initial_probs_np[atg_idx]

0.7748406891668633

Finally, we compute the transition likelihoods. This is where the HMM differs substantially from a simpler approach of using only the most common codons. Here the HMM takes into account the codons that are expected immediately before or after.

In [11]:
transition_counts = np.ones((len(lu_codons), len(lu_codons)))
for gene in all_codons:
    for i in range(0, len(gene)-1):
        codon0 = lu_codons.index(gene[i])
        codon1 = lu_codons.index(gene[i+1])
        transition_counts[codon0, codon1] += 1
transition_totals = transition_counts.sum(axis=1)
transition_probs = np.dot(np.diag(1/transition_totals), transition_counts)
transition_probs_df = pd.DataFrame(transition_probs, index=lu_codons, columns=lu_codons)
transition_probs_df.head()

,AAG,CAA,GCC,GTC,TGG,CCG,GAC,TAC,TGC,TAG,...,ATA,TCA,CCA,CTT,TTC,CAG,ATC,AGA,CGC,TCG
AAG,0.022648,0.028877,0.007998,0.013496,0.011535,0.025647,0.021379,0.008305,0.001961,0.001154,...,0.007575,0.008305,0.008575,0.049333,0.007460,0.034222,0.014035,0.007229,0.009767,0.004230
CAA,0.038904,0.009149,0.016361,0.015908,0.012446,0.002061,0.016237,0.017020,0.004739,0.000989,...,0.010880,0.017433,0.002267,0.003709,0.014795,0.007130,0.036555,0.012034,0.002143,0.009808
GCC,0.018569,0.015059,0.017908,0.020147,0.007123,0.006003,0.012617,0.010887,0.008496,0.000254,...,0.010124,0.009514,0.003154,0.018061,0.011345,0.012108,0.033476,0.017501,0.013635,0.004019
GTC,0.012305,0.010488,0.012445,0.019297,0.007178,0.009089,0.008903,0.006432,0.005593,0.000233,...,0.012259,0.010301,0.003449,0.012678,0.010488,0.008110,0.050480,0.022373,0.011886,0.004148
TGG,0.029720,0.010715,0.009542,0.011028,0.012670,0.016346,0.015251,0.012592,0.004223,0.000704,...,0.012592,0.012670,0.003128,0.032457,0.015799,0.023385,0.027061,0.014860,0.008056,0.006570


#### Build the HMM

In this case, rather than train the HMM (which is [also an option using the `model.fit()` function](https://hmmlearn.readthedocs.io/en/latest/tutorial.html#training-hmm-parameters-and-inferring-the-hidden-states)), we plug in our own calculated matrices to create the model object.

In [12]:
hmm = MultinomialHMM(n_components=len(lu_codons), 
                     startprob_prior=initial_probs_np, 
                     transmat_prior=transition_probs, 
                     verbose=False,  
                     init_params='')
hmm.transmat_ = transition_probs
hmm.emissionprob_ = emission_prob
hmm.startprob_ = initial_probs_np
hmm.n_features = len(lu_aas)

Having built the model, we can sample a randomly-generated sequence and check that everything makes sense.

In [13]:
sample_aa, sample_nuc = hmm.sample(n_samples=10, random_state=21)

In [14]:
assert emission_prob[sample_nuc[0], sample_aa[0]] == 1
decode_seq(sample_aa.reshape(-1), seqtype='prot'), decode_seq(sample_nuc.reshape(-1), seqtype='dna')

('LIENSWEYII', 'TTGATTGAAAATTCCTGGGAATATATTATT')

#### HMM performance

Here we extract a single protein, predict the codon usage, and see how well it compares to the native DNA sequence.

In [15]:
protein = ''
dna = ''
for record in SeqIO.parse(gbk_fn, "genbank"):
    for feature in record.features:
        if feature.type == 'CDS' and 'translation' in feature.qualifiers:
            protein = Seq(feature.qualifiers['translation'][0] + '*')
            dna = feature.extract(record.seq)
            break

In [16]:
protein

Seq('MENILDLWNQALAQIEKKLSKPSFETWMKSTKAHSLQGDTLTITAPNEFARDWL...LK*')

In [17]:
prot_e = encode_seq(protein, seqtype='prot')
mle_dna_indices = hmm.predict(prot_e.reshape(-1, 1))
mle_dna = decode_seq(mle_dna_indices)

Seq(mle_dna).translate()

Seq('MENILDLWNQALAQIEKKLSKPSFETWMKSTKAHSLQGDTLTITAPNEFARDWL...LK*')

In [18]:
def align(seq1, seq2, WIDTH=60):
    '''Align two input sequences of equal length,
    with *  between indicating mismatches.'''
    lines = int(np.ceil(len(seq1) / WIDTH))
    match = ''
    for i, c1 in enumerate(seq1):
        indicator = ' '
        if c1 != seq2[i]:
            indicator = '*'
        match += indicator
    
    for i in range(lines):
        print('Seq1', seq1[i*WIDTH:i*WIDTH+WIDTH])
        print('    ', match[i*WIDTH:i*WIDTH+WIDTH])
        print('Seq2', seq2[i*WIDTH:i*WIDTH+WIDTH])
        print()

align(dna, mle_dna)

Seq1 ATGGAAAATATATTAGACCTGTGGAACCAAGCCCTTGCTCAAATCGAAAAAAAGTTGAGC
                ** *  ** *     *     *           *         *  ***
Seq2 ATGGAAAATATTCTTGATTTATGGAATCAAGCGCTTGCTCAAATTGAAAAAAAGCTGTCA

Seq1 AAACCGAGTTTTGAGACTTGGATGAAGTCAACCAAAGCCCACTCACTGCAAGGCGATACA
          *  *     *  *        ****  *     *  *  *  *     *     *
Seq2 AAACCAAGCTTTGAAACATGGATGAAAAGCACAAAAGCGCATTCTCTTCAAGGAGATACG

Seq1 TTAACAATCACGGCTCCCAATGAATTTGCCAGAGACTGGCTGGAGTCCAGATACTTGCAT
     * *  *  *  *  *  *                       *  *** * *  *  *   
Seq2 CTGACGATTACAGCGCCGAATGAATTTGCCAGAGACTGGCTTGAAAGCCGCTATTTACAT

Seq1 CTGATTGCAGATACTATATATGAATTAACCGGGGAAGAATTGAGCATTAAGTTTGTCATT
     *       *     *  *     ** *  *  *     ** ****     *         
Seq2 TTGATTGCTGATACGATTTATGAGCTGACAGGAGAAGAGCTTTCTATTAAATTTGTCATT

Seq1 CCTCAAAATCAAGATGTTGAGGACTTTATGCCGAAACCGCAAGTCAAAAAAGCGGTCAAA
       *        *        *  *                    *           *   
Seq2 CCGCAAAATCAGGATGTTGAAGATTTTATGCCGAAACCGCAAGTGAAAAAAGCGGTGAAA

Seq1 

In [19]:
state_probabilities = hmm.predict_proba(prot_e.reshape(-1, 1))
state_probabilities[4,]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.1750465 , 0.11127687, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04395898, 0.26055578, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.10726251, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.30189935, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])